### Import Libraries

In [138]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

### Load and Preprocess Data

In [139]:
data = pd.read_csv("random_scores_with_grades.csv")

# Encode the grades
le = LabelEncoder()
data['overAllGrade'] = le.fit_transform(data['overAllGrade'])

# Display the first few rows of the dataframe
data.head()

,index,totalAttendance,quizAverage,assignmentAverage,examAverage,overAllGrade
0,1,17.52,5.38,9.48,31.83,4
1,2,23.77,13.39,10.39,35.41,4
2,3,14.62,12.73,4.08,25.37,4
3,4,83.98,78.54,76.41,89.85,0
4,5,62.12,55.65,63.45,72.96,2


### Define Features and Target

In [140]:
# Features and target
X = data[['totalAttendance', 'quizAverage', 'assignmentAverage', 'examAverage']]
y = data['overAllGrade']

### Split the Data

In [141]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the splits
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 4), (200, 4), (800,), (200,))

### Scale the Features

In [142]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Train the Model

In [143]:
# Train the RandomForestClassifier
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=42)

### Evaluate the Model

In [144]:
# Predict and evaluate
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.91
              precision    recall  f1-score   support

           A       0.95      0.97      0.96        38
           B       0.94      0.80      0.86        20
           C       0.82      1.00      0.90        49
           D       0.00      0.00      0.00        13
           F       0.95      1.00      0.98        80

    accuracy                           0.91       200
   macro avg       0.73      0.75      0.74       200
weighted avg       0.86      0.91      0.88       200



d:\Study\WEB\Projects\btp\model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Study\WEB\Projects\btp\model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Study\WEB\Projects\btp\model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Predict Graduation Likelihood

In [145]:
# Example prediction
example_scores = {
    'totalAttendance':100,
    'quizAverage': 100,
    'assignmentAverage': 5,
    'examAverage': 50
}

example_scores_scaled = scaler.transform([list(example_scores.values())])
predicted_grade_index = model.predict(example_scores_scaled)[0]
predicted_grade = le.inverse_transform([predicted_grade_index])[0]

print("Predicted Grade:", predicted_grade)

Predicted Grade: C


d:\Study\WEB\Projects\btp\model\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


### Comparision Plot

In [146]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Define customizable parameters
actual_color = 'blue'
same_color = 'green'
different_color = 'red'
actual_alpha = 0.6
same_alpha = 0.6
different_alpha = 1
same_size = 40
different_size = 100

# Assuming you have the actual and predicted values
y_test_pred = model.predict(X_test_scaled)

# Create the DataFrame for plotting
comparison_df = pd.DataFrame({
    'Index': range(len(y_test)),
    'Actual Grade': le.inverse_transform(y_test),
    'Predicted Grade': le.inverse_transform(y_test_pred)
})

# Determine colors and sizes for predicted grades
comparison_df['Color'] = np.where(comparison_df['Actual Grade'] == comparison_df['Predicted Grade'], same_color, different_color)
comparison_df['Size'] = np.where(comparison_df['Color'] == different_color, different_size, same_size)
comparison_df['Alpha'] = np.where(comparison_df['Color'] == different_color, different_alpha, same_alpha)

plt.figure(figsize=(18, 6))

# Plot Actual Grades
sns.scatterplot(data=comparison_df, x='Index', y='Actual Grade', color=actual_color, label='Actual Grade', alpha=actual_alpha, edgecolor='w')

# Plot Predicted Grades with manual color and alpha assignment
for color in comparison_df['Color'].unique():
    subset = comparison_df[comparison_df['Color'] == color]
    label = 'Same' if color == same_color else 'Different'
    sns.scatterplot(data=subset, x='Index', y='Predicted Grade', color=color, label=label, alpha=subset['Alpha'].iloc[0], edgecolor='w', s=subset['Size'])

plt.title('Scatter Plot of Actual vs Predicted Grades')
plt.xlabel('Index')
plt.ylabel('Grade')
plt.legend()

plt.show()

In [147]:
import joblib

joblib.dump(model, 'model_LR.pkl')

['model_LR.pkl']